In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from termcolor import colored
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalMaxPooling1D, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import wandb
from wandb.keras import WandbCallback
wandb.login()
WANDB_CONFIG = {'competition': 'AG News Classification Dataset', '_wandb_kernel': 'neuracort'}
TRAIN_FILE_PATH = 'dataset/train.csv'
TEST_FILE_PATH = 'dataset/test.csv'
data = pd.read_csv(TRAIN_FILE_PATH)
testdata = pd.read_csv(TEST_FILE_PATH)
data.columns = ['ClassIndex', 'Title', 'Description']
testdata.columns = ['ClassIndex', 'Title', 'Description']
X_train = data['Title'] + " " + data['Description']
y_train = data['ClassIndex'].apply(lambda x: x-1).values
x_test = testdata['Title'] + " " + testdata['Description']
y_test = testdata['ClassIndex'].apply(lambda x: x-1).values
maxlen = X_train.map(lambda x: len(x.split())).max()
vocab_size = 10000
embed_size = 32
tok = Tokenizer(num_words=vocab_size)
tok.fit_on_texts(X_train.values)
X_train = tok.texts_to_sequences(X_train)
x_test = tok.texts_to_sequences(x_test)
X_train = pad_sequences(X_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
run = wandb.init(project='ag-news', config=WANDB_CONFIG)
model = Sequential([
    Embedding(vocab_size, embed_size, input_length=maxlen),
    Bidirectional(LSTM(128, return_sequences=True)),
    Bidirectional(LSTM(64, return_sequences=True)),
    GlobalMaxPooling1D(),
    Dense(1024),
    Dropout(0.25),
    Dense(512),
    Dropout(0.25),
    Dense(256),
    Dropout(0.25),
    Dense(128),
    Dropout(0.25),
    Dense(64),
    Dropout(0.25),
    Dense(4, activation='softmax')
])
callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        min_delta=1e-4,
        patience=4,
        verbose=1
    ),
    ModelCheckpoint(
        filepath='weights.h5',
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        save_weights_only=True,
        verbose=1
    ),
    WandbCallback()
]

C:\Users\venka\AppData\Local\Temp\ipykernel_15676\4039476188.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: venkatachaitanya373 (chaitanya8). Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [2]:
model.load_weights('weights.h5')
model.save('model.hdf5')

def modelDemo(news_text):
    labels = ['World News', 'Sports News', 'Business News', 'Science-Technology News']
    test_seq = pad_sequences(tok.texts_to_sequences(news_text), maxlen=maxlen)
    test_preds = [labels[np.argmax(i)] for i in model.predict(test_seq)]
    for news, label in zip(news_text, test_preds):
        return('{} - {}'.format(colored(news, 'green'), colored(label, 'white')))

c:\Users\venka\Desktop\ML NEWS V2\venv_news_class\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# DEMO:

In [3]:
print(modelDemo(['New evidence of virus risks from wildlife trade']))

1/1 [==============================] - 2s 2s/step
New evidence of virus risks from wildlife trade - Science-Technology News


In [4]:
print(modelDemo(['Coronavirus: Bank pumps £100bn into UK economy to aid recovery']))

1/1 [==============================] - 0s 66ms/step
Coronavirus: Bank pumps £100bn into UK economy to aid recovery - Business News


In [5]:
print(modelDemo(['Trump\'s bid to end Obama-era immigration policy ruled unlawful']))

1/1 [==============================] - 0s 61ms/step
Trump's bid to end Obama-era immigration policy ruled unlawful - World News


In [6]:
print(modelDemo(['David Luiz’s future with Arsenal to be decided this week']))

1/1 [==============================] - 0s 63ms/step
David Luiz’s future with Arsenal to be decided this week - Sports News


In [7]:
print(modelDemo(['Indian Economic budget supports the underprivileged sections of society']))

1/1 [==============================] - 0s 70ms/step
Indian Economic budget supports the underprivileged sections of society - Business News
